# Aim
Read all csv files generated by the twint analysis. Twint analysis did only return, tweets but not user location. Do get user location, read the csv and query each user and extract the location. <br>
Once the location has been extracted, the location has to be mapped to a US State.

# Important informations for States

In [ ]:
# Important informations
StatesFull = ['Alabama','Alaska','Arizona','Arkansas','California', 'Colorado','Connecticut',
          'Delaware','Florida','Georgia','Hawaii','Idaho','Illinois','Indiana','Iowa',
          'Kansas','Kentucky','Louisiana','Maine','Maryland','Massachusetts','Michigan','Minnesota',
          'Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire','New Jersey',
          'New Mexico','New York','North Carolina','North Dakota','Ohio','Oklahoma','Oregon',
          'Pennsylvania','Rhode Island','South Carolina','South Dakota','Tennessee','Texas','Utah',
          'Vermont','Virginia','Washington','West Virginia','Wisconsin','Wyoming']
StatesFull=[S.lower() for S in StatesFull]
print(len(StatesFull))
StatesTL=['AL','AK','AZ','AR','CA','CO','CT','DE','FL','KY','LA','ME','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK',
         'OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY','KS','GA','HI','MD','MA','ID','IL','IN','MI',
          'IA','MN','MS','MO']
StatesTL=[S.lower() for S in StatesTL]
print(len(StatesTL))
States_dict={'Alabama' : 'AL','Alaska' : 'AK','Arizona' : 'AZ','Arkansas' : 'AR','California': 'CA','Colorado' : 'CO',
             'Connecticut' : 'CT','Delaware' : 'DE','Florida' : 'FL','Georgia' : 'GA','Hawaii' : 'HI','Idaho' : 'ID',
             'Illinois' : 'IL','Indiana' : 'IN','Iowa' : 'IA', 'Kansas' : 'KS','Kentucky' : 'KY','Louisiana' : 'LA','Maine':'ME',
             'Maryland' : 'MD','Massachusetts': 'MA' ,'Michigan' : 'MI','Minnesota' : 'MN', 'Mississippi' : 'MS','Missouri' : 'MO',
             'Montana' : 'MT','Nebraska' : 'NE','Nevada' : 'NV','New hampshire' : 'NH','New jersey' : 'NJ','New mexico' : 'NM',
             'New york' : 'NY', 'North carolina' : 'NC','North dakota' : 'ND', 'Ohio' : 'OH', 'Oklahoma' : 'OK','Oregon' : 'OR',
             'Pennsylvania' : 'PA', 'Rhode island' : 'RI','South carolina' : 'SC','South dakota' : 'SD','Tennessee' : 'TN',
             'Texas' : 'TX', 'Utah' : 'UT', 'Vermont' : 'VT', 'Virginia' : 'VA','Washington' : 'WA', 'West virginia' : 'WV',
             'Wisconsin' :'WI', 'Wyoming':'WY'}
print(len(States_dict))

# Process

In [ ]:
import tweepy
import csv
import json
import pandas as pd
from collections import Counter

In [ ]:
# create a dictionary to store your twitter credentials

twitter_cred = dict()

# Enter your own consumer_key, consumer_secret, access_key and access_secret
# Replacing the stars ("********")

twitter_cred['CONSUMER_KEY'] = ''
twitter_cred['CONSUMER_SECRET'] = ''
twitter_cred['ACCESS_KEY'] = ''
twitter_cred['ACCESS_SECRET'] = ''

# Save the information to a json so that it can be reused in code without exposing
# the secret info to public

with open('twitter_credentials.json', 'w') as secret_info:
    json.dump(twitter_cred, secret_info, indent=4, sort_keys=True)

In [ ]:
# load Twitter API credentials
with open('twitter_credentials.json') as cred_data:
    info = json.load(cred_data)
    consumer_key = info['CONSUMER_KEY']
    consumer_secret = info['CONSUMER_SECRET']
    access_key = info['ACCESS_KEY']
    access_secret = info['ACCESS_SECRET']

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
# SPlit the content of the location column using a co
# Check if the Location contains the two-letter state code 
USNames=['united states', 'usa', 'us']
def determine_USState(all_tweets):
    StatesTest=[]
    for i,L in enumerate(all_tweets['Location']):
        L=L.strip().lower()
        if ',' in L: ## Orlando, FL
            test=L.split(',')[1] 
            test=test.strip()
            if test in StatesTL:
                StatesTest.append(test.capitalize())
            elif test in StatesFull:
                StatesTest.append(States_dict[test.capitalize()])
            elif test in USNames: # For these cases Florida, USA
                firstelement=L.split(',')[0]
                if firstelement in StatesTL:
                    StatesTest.append(firstelement.capitalize())
                elif firstelement in StatesFull:
                    StatesTest.append(States_dict[firstelement.capitalize()])
                else:
                    StatesTest.append('probably not US')
                    next  
            else:
                StatesTest.append('probably not US')
                next           
        elif L in StatesFull:
            StatesTest.append(States_dict[L.capitalize()])
        else:
            StatesTest.append('probably not US')

    all_tweets['State']=StatesTest
    all_tweets['State']=all_tweets['State'].str.lower()
    return(all_tweets)

In [ ]:
# loop through each individual csv and 
import glob
path = "Foods2016\*.csv"
for fname in glob.glob(path):
    print(fname)
    CSV_name=fname.split('\\')[1]
    Cat=CSV_name.split('_')[0]
    Query=CSV_name.split('_')[1]
    Year=CSV_name.split('_')[2].split('-')[0]
    
    f= pd.read_csv(fname)
    usernames=f.username
    userlocations=[]
    for user in usernames:
        try:
            userdata=api.get_user(user)
            userlocations.append(userdata.location)
        except: 
            print(user)
            #print("Unexpected error:", sys.exc_info()[0])
            userlocations.append('')
    f['Cat']=Cat
    f['Query']=Query
    f['Year']=Year
    f['Location']=userlocations
    f=determine_USState(f)
    
    newfilename='WithLocation2016\Loc_'+CSV_name
    f.to_csv(newfilename)